<a href="https://colab.research.google.com/github/jseverina/RN-TP2/blob/main/07_lightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [2]:
import pandas as pd
import numpy as np
import datetime
!pip install pandas_summary
from pandas_summary import DataFrameSummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 4.3 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/Redes Neuronales/TP2/'
df = pd.read_feather(path + 'train_normalized_data.fth')
df_test = pd.read_feather(path + 'test_normalized_data.fth')

In [5]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [8]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [9]:
from lightgbm import LGBMRegressor

In [10]:
y_out_columns = ['Sales']

In [11]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [12]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [13]:
X_train.shape, X_val.shape
# (814150, 38), (30188, 38))

((814150, 36), (30188, 36))

In [14]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [15]:
min_child_samples=5
n_estimators=4000
learning_rate=0.05
model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate )

In [16]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'l2', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }

In [17]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds.
[100]	valid's l2: 0.000214639	valid's l2: 0.000214639
[200]	valid's l2: 0.000160008	valid's l2: 0.000160008
[300]	valid's l2: 0.000143308	valid's l2: 0.000143308
[400]	valid's l2: 0.000137622	valid's l2: 0.000137622
[500]	valid's l2: 0.000131873	valid's l2: 0.000131873
[600]	valid's l2: 0.000128251	valid's l2: 0.000128251
[700]	valid's l2: 0.000125225	valid's l2: 0.000125225
[800]	valid's l2: 0.000122846	valid's l2: 0.000122846
[900]	valid's l2: 0.000119729	valid's l2: 0.000119729
[1000]	valid's l2: 0.000117034	valid's l2: 0.000117034
[1100]	valid's l2: 0.000116105	valid's l2: 0.000116105
[1200]	valid's l2: 0.000115506	valid's l2: 0.000115506
[1300]	valid's l2: 0.000114338	valid's l2: 0.000114338
[1400]	valid's l2: 0.000113653	valid's l2: 0.000113653
[1500]	valid's l2: 0.000113283	valid's l2: 0.000113283
[1600]	valid's l2: 0.000113092	valid's l2: 0.000113092
[1700]	valid's l2: 0.000112761	valid's l2: 0.000112761
[1800]	vali

LGBMRegressor(learning_rate=0.05, min_child_samples=5, n_estimators=4000)

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [18]:
model.score(X_val, y_val)
# 0.9193971359680227

0.9210727178615297

In [19]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [20]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))
# 0.0817797546225218

0.09086254402968043

In [21]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))
# 0.11897277749253428

0.11665364339590643

# Baseline

In [22]:
import pandas as pd
sample_csv = pd.read_csv(path + 'dataset/sample_submission.csv')

In [23]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [24]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [25]:
df_test[['Store', 'Sales']].head(10)

# ta igual

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [26]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [27]:
sample_csv['Sales'] = df_test['Sales']

In [28]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [29]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [30]:
import shutil
shutil.copy("/content/submision_baseline.csv","/content/drive/MyDrive/Colab Notebooks/Redes Neuronales/TP2")

'/content/drive/MyDrive/Colab Notebooks/Redes Neuronales/TP2/submision_baseline.csv'

In [31]:
sample_csv = pd.read_csv(path + 'dataset/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv', index=False)

In [32]:
import shutil
shutil.copy(f'/content/submision_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv',"/content/drive/MyDrive/Colab Notebooks/Redes Neuronales/TP2")

'/content/drive/MyDrive/Colab Notebooks/Redes Neuronales/TP2/submision_True-5-4000-0.05.csv'

In [33]:
import shutil
shutil.copy(f'/content/submision_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv',"/content/drive/MyDrive/Colab Notebooks/Redes Neuronales/TP2")

'/content/drive/MyDrive/Colab Notebooks/Redes Neuronales/TP2/submision_True-5-4000-0.05.csv'